In [1]:
import os
path = os.getcwd()
if path == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    BASE_PATH = '/content/gdrive/My Drive/Level-4-Project/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
elif path == 'D:\\Google Drive\\Level-4-Project\\notebooks\\data_processing\\wavenet':
    BASE_PATH = "D:/Google Drive/Level-4-Project/"
    
elif path == "/export/home/2192793m":
    BASE_PATH = "/export/home/2192793m/Level-4-Project/"
    

DATA_PATH = BASE_PATH + 'data/'
RAW_PATH = DATA_PATH + 'raw/raw_converted/'
PROCESSED_PATH = DATA_PATH + 'processed/wavenet/'
    
from src.data import load_data
from src.visualization import multiple_plots, visualize, plot_confusion_matrix
from src.features import make_spectrograms, process_labels, make_directory

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Obtaining file:///content/gdrive/My%20Drive/Level-4-Project
  Found existing installation: src 0.1.0
    Can't uninstall 'src'. No files were found to uninstall.
  Running setup.py develop for src


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import mlab
from matplotlib import colors
from scipy.signal import butter, freqz, lfilter, spectrogram
import time
from sklearn import preprocessing

In [3]:
import numpy as np
import sys
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
import sys
from sklearn.metrics import classification_report, confusion_matrix
import csv
from keras.models import load_model
import pickle

Using TensorFlow backend.


In [0]:
labels = ["walking", "pushing", "sitting", "pulling", "circling", "clapping", "bending"]
processed = {"A":{}, "B":{}, "C":{}, "D":{}, "E":{}, "F":{}}
for name, item in processed.items():
    for label in labels:
        item[label] = []

In [0]:
def nearest_odd_number(x):
    if(np.floor(x) % 2 == 0):
        return int(np.floor(x) + 1)
    else:
        return int(np.floor(x))

In [0]:
def create_range_fft(reshaped_data):
    win = np.ones(reshaped_data.shape)
    # Apply fast fourier transform should compute distance (range) from objects
    fft_applied = np.fft.fft((reshaped_data * win), axis=0)
    return fft_applied[:int(number_of_time_samples/2), :] # take half for some reason?

In [0]:
def MTI_filter(range_fft):
    # IIR Notch filter
    x = range_fft.shape[1]
    ns = nearest_odd_number(x) - 1
    data_range_MTI = np.zeros((range_fft.shape[0], ns), dtype=np.complex128)
    (b, a) = butter(4, 0.01, btype="high")
    # Apply Filter
    for i in range(range_fft.shape[0]):
        data_range_MTI[i, :ns] = lfilter(b, a, range_fft[i, :ns], axis=0)
    # Remove first range bin as has strong residual possibly from filtering?
    return data_range_MTI[1:, :]

In [0]:
def create_doppler_fft(range_fft):
    # Selects range bins
    bin_indl = 5
    bin_indu = 25

    prf = 1/sweep_time
    time_window_length = 200
    overlap_factor = 0.95
    overlap_length = np.round(time_window_length * overlap_factor)
    pad_factor = 4
    fft_points = pad_factor * time_window_length
    doppler_bin = prf / fft_points
    doppler_axis = np.arange(-prf / 2, prf / 2 - doppler_bin + 1, doppler_bin)
    whole_duration = range_fft.shape[1] / prf
    num_segments = np.floor((range_fft.shape[1] - time_window_length) /
        (np.floor(time_window_length * (1 - overlap_factor))))
    
    data_spec_MTI2=0
    for rbin in range(bin_indl-1, bin_indu):    
        s, f, t = mlab.specgram(range_fft[rbin, :],
                                Fs=1,
                                window=np.hamming(time_window_length),
                                noverlap=overlap_length,
                                NFFT=time_window_length, 
                                mode='complex',
                                pad_to=fft_points )

        data_MTI_temp = np.fft.fftshift(s, 1)
        data_spec_MTI2=data_spec_MTI2+data_MTI_temp  
    return data_spec_MTI2

In [0]:
df_labels = pd.read_csv(RAW_PATH + 'Labels.csv')
df_labels.rename(columns={'dataset ID':'dataset_id'}, inplace=True)

In [0]:
df_labels = process_labels.process_labels(df_labels)

In [11]:
number_of_rows = df_labels.shape[0]
current_row = 1
for row in df_labels.itertuples():
    print(row.user_label, row.aspect_angle, row.label)
    if row.aspect_angle != "0":
        current_row += 1
        continue
        
    print("Processing row", current_row, "of", number_of_rows)
    file_name = RAW_PATH + "Dataset_" + str(row.dataset_id) + ".dat"
    radar_df = pd.read_csv(file_name, header=None)[1]
    
    # Grab RADAR settings from top of file
    center_frequency = float(radar_df.iloc[1])
    sweep_time = float(radar_df.iloc[2])/1000  # convert to seconds
    number_of_time_samples = float(radar_df.iloc[3])
    bandwidth = float(radar_df.iloc[4])
    sampling_frequency = number_of_time_samples/sweep_time
    record_length = 60
    number_of_chirps = record_length/sweep_time
    
    # Put data values into an array
    data = radar_df.iloc[5:].apply(complex).values
    
    # Reshape into chirps over time
    data_time = np.reshape(data, (int(number_of_chirps),int(number_of_time_samples)))
    data_time = np.rot90(data_time, k=-1)
    # compute range fft
    range_fft = create_range_fft(data_time)
    # Apply MTI filter to remove stationary objects
    MTI_filter_applied = MTI_filter(range_fft)[0:35, :]
                                              
    window_length = 3                      
    window_size = int(window_length * 1000)
    iterations = MTI_filter_applied.shape[1] - window_size
    step_size = 1000  # 1 seconds (was orignaly 10)
    spectrograms = []
    for i in range(0, iterations, step_size):
        spectrograms.append(MTI_filter_applied[:,i:(i + window_size):2]) # take every second column to keep total size down
#         plt.imshow(20 * np.log10(abs(MTI_filter_applied[:,i:(i + window_size):2])), cmap="jet", aspect='auto')
#         plt.show()
        
    processed[row.user_label][row.label].append(spectrograms)

    current_row += 1

A 0 walking
Processing row 1 of 123
A 0 walking
Processing row 2 of 123
A 30 walking
A 30 walking
A 60 walking
A 60 walking
A 0 pushing
Processing row 7 of 123
A 0 pushing
Processing row 8 of 123
A 30 pushing
A 30 pushing
A 60 pushing
A 60 pushing
A 0 sitting
Processing row 13 of 123
A 0 sitting
Processing row 14 of 123
A 0 pulling
Processing row 15 of 123
A 0 pulling
Processing row 16 of 123
A 0 circling
Processing row 17 of 123
A 0 circling
Processing row 18 of 123
A 0 clapping
Processing row 19 of 123
A 0 clapping
Processing row 20 of 123
A 0 bending
Processing row 21 of 123
A 0 bending
Processing row 22 of 123
B 0 walking
Processing row 23 of 123
B 0 walking
Processing row 24 of 123
B 0 pushing
Processing row 25 of 123
B 0 pushing
Processing row 26 of 123
B 0 sitting
Processing row 27 of 123
B 0 sitting
Processing row 28 of 123
B 0 circling
Processing row 29 of 123
B 0 circling
Processing row 30 of 123
B 0 clapping
Processing row 31 of 123
B 0 clapping
Processing row 32 of 123
B 0 

In [0]:
with open(PROCESSED_PATH + "range_fft_MTI.pkl", "wb") as file:
    pickle.dump(processed, file)

In [1]:
!ls -lah data/processed/wavenet

ls: cannot access 'data/processed/wavenet': No such file or directory
